In [1]:
import pandas as pd;
import numpy as np;
from sklearn.preprocessing import LabelEncoder;
from sklearn.tree import DecisionTreeClassifier;
from sklearn import tree

In [2]:
TRAINING_FILE = "crime_train.csv";
TEST_FILE = "crime_test.csv";

In [3]:
df_train = pd.DataFrame.from_csv(TRAINING_FILE, parse_dates=True, index_col=None);
df_test = pd.DataFrame.from_csv(TEST_FILE, parse_dates=True, index_col=None);

C:\Users\sajan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.
C:\Users\sajan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562484 entries, 0 to 562483
Data columns (total 9 columns):
Dates         562484 non-null object
Category      562484 non-null object
Descript      562484 non-null object
DayOfWeek     562484 non-null object
PdDistrict    562484 non-null object
Resolution    562484 non-null object
Address       562484 non-null object
X             562484 non-null float64
Y             562484 non-null float64
dtypes: float64(2), object(7)
memory usage: 38.6+ MB


In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140621 entries, 0 to 140620
Data columns (total 7 columns):
id            140621 non-null int64
Dates         140621 non-null object
DayOfWeek     140621 non-null object
PdDistrict    140621 non-null object
Address       140621 non-null object
X             140621 non-null float64
Y             140621 non-null float64
dtypes: float64(2), int64(1), object(4)
memory usage: 7.5+ MB


In [6]:
df_train = df_train.drop(['Descript', 'Resolution'], axis=1)

In [7]:
def feature_engineering(data):
    data['Day'] = data['customdate'].dt.day
    data['Month'] = data['customdate'].dt.month
    data['Year'] = data['customdate'].dt.year
    data['Hour'] = data['customdate'].dt.hour
    data['Minute'] = data['customdate'].dt.minute
    data['DayOfWeekEnc'] = data['customdate'].dt.dayofweek
    data['WeekOfYear'] = data['customdate'].dt.weekofyear
    return data


In [8]:
df_train['customdate'] = pd.to_datetime(df_train['Dates'], format = "%Y-%m-%d %H:%M:%S")

In [9]:
df_train['Day'] = df_train['customdate'].dt.day
df_train['Month'] = df_train['customdate'].dt.month
df_train['Year'] = df_train['customdate'].dt.year
df_train['Hour'] = df_train['customdate'].dt.hour
df_train['DayOfWeek'] = df_train['customdate'].dt.dayofweek
df_train['WeekOfYear'] = df_train['customdate'].dt.weekofyear

In [10]:
df_train['isIntersection'] = pd.np.where(df_train.Address.str.contains("/"),1,0)
df_train['isBlock'] = pd.np.where(df_train.Address.str.contains("Block"),1,0)

In [11]:
df_train

,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y,customdate,Day,Month,Year,Hour,WeekOfYear,isIntersection,isBlock
0,2010-09-12 14:52:00,OTHER OFFENSES,6,RICHMOND,2900 Block of BALBOA ST,-122.490357,37.775978,2010-09-12 14:52:00,12,9,2010,14,36,0,1
1,2011-04-23 22:00:00,LARCENY/THEFT,5,SOUTHERN,0 Block of ELGINPARK ST,-122.423155,37.770642,2011-04-23 22:00:00,23,4,2011,22,16,0,1
2,2008-02-20 16:45:00,MISSING PERSON,2,MISSION,900 Block of POTRERO AV,-122.406870,37.757290,2008-02-20 16:45:00,20,2,2008,16,8,0,1
3,2014-10-03 16:00:00,VANDALISM,4,RICHMOND,400 Block of JOHNFKENNEDY DR,-122.465209,37.772610,2014-10-03 16:00:00,3,10,2014,16,40,0,1
4,2012-06-11 09:04:00,OTHER OFFENSES,0,RICHMOND,3600 Block of CALIFORNIA ST,-122.454102,37.786385,2012-06-11 09:04:00,11,6,2012,9,24,0,1
5,2009-06-27 09:00:00,VEHICLE THEFT,5,CENTRAL,PINE ST / POWELL ST,-122.408978,37.791089,2009-06-27 09:00:00,27,6,2009,9,26,1,0
6,2014-11-28 21:00:00,VANDALISM,4,MISSION,700 Block of VALENCIA ST,-122.421431,37.760890,2014-11-28 21:00:00,28,11,2014,21,48,0,1
7,2007-11-27 20:30:00,ASSAULT,1,BAYVIEW,1400 Block of RANKIN ST,-122.398722,37.737928,2007-11-27 20:30:00,27,11,2007,20,48,0,1
8,2013-08-21 03:15:00,OTHER OFFENSES,2,SOUTHERN,5TH ST / CLEMENTINA ST,-122.403924,37.780699,2013-08-21 03:15:00,21,8,2013,3,34,1,0
9,2009-03-06 23:30:00,LARCENY/THEFT,4,NORTHERN,3100 Block of FILLMORE ST,-122.435796,37.798257,2009-03-06 23:30:00,6,3,2009,23,10,0,1


In [12]:
df_test['customdate'] = pd.to_datetime(df_test['Dates'], format = "%Y-%m-%d %H:%M:%S")

In [13]:
df_test['Day'] = df_test['customdate'].dt.day
df_test['Month'] = df_test['customdate'].dt.month
df_test['Year'] = df_test['customdate'].dt.year
df_test['Hour'] = df_test['customdate'].dt.hour
df_test['DayOfWeek'] = df_test['customdate'].dt.dayofweek
df_test['WeekOfYear'] = df_test['customdate'].dt.weekofyear

In [14]:
df_test['isIntersection'] = pd.np.where(df_test.Address.str.contains("/"),1,0)
df_test['isBlock'] = pd.np.where(df_test.Address.str.contains("Block"),1,0)

In [15]:
df_test

,id,Dates,DayOfWeek,PdDistrict,Address,X,Y,customdate,Day,Month,Year,Hour,WeekOfYear,isIntersection,isBlock
0,1,2009-11-23 18:10:00,0,SOUTHERN,800 Block of BRYANT ST,-122.403405,37.775421,2009-11-23 18:10:00,23,11,2009,18,48,0,1
1,2,2013-04-29 09:00:00,0,TARAVAL,1200 Block of OCEAN AV,-122.456312,37.724006,2013-04-29 09:00:00,29,4,2013,9,18,0,1
2,3,2007-10-20 11:20:00,5,SOUTHERN,MCALLISTER ST / JONES ST,-122.411979,37.780904,2007-10-20 11:20:00,20,10,2007,11,42,1,0
3,4,2007-06-16 18:38:00,5,NORTHERN,1000 Block of VANNESS AV,-122.421092,37.785000,2007-06-16 18:38:00,16,6,2007,18,24,0,1
4,5,2013-04-29 23:30:00,0,INGLESIDE,1000 Block of MUNICH ST,-122.434673,37.711055,2013-04-29 23:30:00,29,4,2013,23,18,0,1
5,6,2013-07-24 00:01:00,2,CENTRAL,STOCKTON ST / GEARY ST,-122.406592,37.787567,2013-07-24 00:01:00,24,7,2013,0,30,1,0
6,7,2007-12-10 20:36:00,0,CENTRAL,1400 Block of TAYLOR ST,-122.413347,37.795641,2007-12-10 20:36:00,10,12,2007,20,50,0,1
7,8,2008-07-25 10:51:00,4,TENDERLOIN,TAYLOR ST / TURK ST,-122.410769,37.783215,2008-07-25 10:51:00,25,7,2008,10,30,1,0
8,9,2007-08-22 12:00:00,2,SOUTHERN,800 Block of BRYANT ST,-122.403405,37.775421,2007-08-22 12:00:00,22,8,2007,12,34,0,1
9,10,2012-05-04 11:00:00,4,TENDERLOIN,0 Block of TURK ST,-122.409849,37.783276,2012-05-04 11:00:00,4,5,2012,11,18,0,1


In [16]:
#encoder to convert PdDistrict categorical value to a numerical value
train_encoder = LabelEncoder()
df_train['PdDistrict'] = train_encoder.fit_transform(df_train['PdDistrict'])

#encoder to convert PdDistrict categorical value to a numerical value
test_encoder = LabelEncoder()
df_test['PdDistrict'] = test_encoder.fit_transform(df_test['PdDistrict'])

In [17]:
category_encoder = LabelEncoder() 
category_encoder.fit(df_train['Category']) 
df_train['CategoryEncoded'] = category_encoder.transform(df_train['Category']) 
print(category_encoder.classes_)

['ARSON' 'ASSAULT' 'BAD CHECKS' 'BRIBERY' 'BURGLARY' 'DISORDERLY CONDUCT'
 'DRIVING UNDER THE INFLUENCE' 'DRUG/NARCOTIC' 'DRUNKENNESS'
 'EMBEZZLEMENT' 'EXTORTION' 'FAMILY OFFENSES' 'FORGERY/COUNTERFEITING'
 'FRAUD' 'GAMBLING' 'KIDNAPPING' 'LARCENY/THEFT' 'LIQUOR LAWS' 'LOITERING'
 'MISSING PERSON' 'NON-CRIMINAL' 'OTHER OFFENSES'
 'PORNOGRAPHY/OBSCENE MAT' 'PROSTITUTION' 'RECOVERED VEHICLE' 'ROBBERY'
 'RUNAWAY' 'SECONDARY CODES' 'SEX OFFENSES FORCIBLE'
 'SEX OFFENSES NON FORCIBLE' 'STOLEN PROPERTY' 'SUICIDE' 'SUSPICIOUS OCC'
 'TREA' 'TRESPASS' 'VANDALISM' 'VEHICLE THEFT' 'WARRANTS' 'WEAPON LAWS']


In [18]:
print(df_train.columns)
print(df_test.columns)

Index(['Dates', 'Category', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y',
       'customdate', 'Day', 'Month', 'Year', 'Hour', 'WeekOfYear',
       'isIntersection', 'isBlock', 'CategoryEncoded'],
      dtype='object')
Index(['id', 'Dates', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y',
       'customdate', 'Day', 'Month', 'Year', 'Hour', 'WeekOfYear',
       'isIntersection', 'isBlock'],
      dtype='object')


In [19]:
df_train = df_train.drop(['customdate'],axis=1)
df_test = df_test.drop(['customdate'], axis=1)

In [20]:
df_train = df_train.drop(['Address'],axis=1)
df_test = df_test.drop(['Address'], axis=1)

In [21]:
attr_cols = list(df_train.columns[2:-1])
print(attr_cols)

['DayOfWeek', 'PdDistrict', 'X', 'Y', 'Day', 'Month', 'Year', 'Hour', 'WeekOfYear', 'isIntersection', 'isBlock']


In [38]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(df_train[attr_cols], df_train['CategoryEncoded'])
predictions = np.array(clf.predict_proba(df_test[attr_cols]))

In [42]:
predictions
predictions.shape

(140621, 39)

In [52]:
result = pd.DataFrame(predictions, columns=category_encoder.classes_)
result.to_csv('test_sample_submission.csv', index = True, index_label = 'id' )

In [25]:
from sklearn.metrics import log_loss

In [26]:
def field_to_columns(data, field, new_columns):
    for i in range(len(new_columns)):
        data[new_columns[i]] = data[field]
    return data

In [27]:
df_test.columns

Index(['id', 'Dates', 'DayOfWeek', 'PdDistrict', 'X', 'Y', 'Day', 'Month',
       'Year', 'Hour', 'WeekOfYear', 'isIntersection', 'isBlock',
       'predictions'],
      dtype='object')

In [28]:
df_test['Category'] = category_encoder.inverse_transform(df_test['predictions'])

C:\Users\sajan\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [29]:
categories = list(category_encoder.classes_)

In [30]:
df_test['Category']

0                  LARCENY/THEFT
1                  LARCENY/THEFT
2                 OTHER OFFENSES
3                       WARRANTS
4                   NON-CRIMINAL
5                  LARCENY/THEFT
6                   NON-CRIMINAL
7                 OTHER OFFENSES
8                   NON-CRIMINAL
9                   NON-CRIMINAL
10                      TRESPASS
11                       ASSAULT
12                  NON-CRIMINAL
13                      WARRANTS
14                  NON-CRIMINAL
15                       ASSAULT
16                 LARCENY/THEFT
17                 LARCENY/THEFT
18                 DRUG/NARCOTIC
19                      BURGLARY
20                   WEAPON LAWS
21                      WARRANTS
22                      BURGLARY
23                 VEHICLE THEFT
24                 VEHICLE THEFT
25                 DRUG/NARCOTIC
26                 LARCENY/THEFT
27                   LIQUOR LAWS
28                         FRAUD
29                 VEHICLE THEFT
          

In [31]:
categories = list(category_encoder.classes_)

In [32]:
df_test = field_to_columns(df_test, 'Category', categories)

In [33]:
import time
PREDICTIONS_FILENAME_PREFIX = 'predictions_'
PREDICTIONS_FILENAME = PREDICTIONS_FILENAME_PREFIX + time.strftime('%Y%m%d-%H%M%S') + '.csv'

In [34]:
print(df_test.columns)

Index(['id', 'Dates', 'DayOfWeek', 'PdDistrict', 'X', 'Y', 'Day', 'Month',
       'Year', 'Hour', 'WeekOfYear', 'isIntersection', 'isBlock',
       'predictions', 'Category', 'ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY',
       'BURGLARY', 'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE',
       'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION',
       'FAMILY OFFENSES', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING',
       'KIDNAPPING', 'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING',
       'MISSING PERSON', 'NON-CRIMINAL', 'OTHER OFFENSES',
       'PORNOGRAPHY/OBSCENE MAT', 'PROSTITUTION', 'RECOVERED VEHICLE',
       'ROBBERY', 'RUNAWAY', 'SECONDARY CODES', 'SEX OFFENSES FORCIBLE',
       'SEX OFFENSES NON FORCIBLE', 'STOLEN PROPERTY', 'SUICIDE',
       'SUSPICIOUS OCC', 'TREA', 'TRESPASS', 'VANDALISM', 'VEHICLE THEFT',
       'WARRANTS', 'WEAPON LAWS'],
      dtype='object')


In [35]:
submission_cols = [df_test.columns[0]]+list(df_test.columns[14:])
print(submission_cols)

['id', 'Category', 'ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY', 'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE', 'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION', 'FAMILY OFFENSES', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING', 'KIDNAPPING', 'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING', 'MISSING PERSON', 'NON-CRIMINAL', 'OTHER OFFENSES', 'PORNOGRAPHY/OBSCENE MAT', 'PROSTITUTION', 'RECOVERED VEHICLE', 'ROBBERY', 'RUNAWAY', 'SECONDARY CODES', 'SEX OFFENSES FORCIBLE', 'SEX OFFENSES NON FORCIBLE', 'STOLEN PROPERTY', 'SUICIDE', 'SUSPICIOUS OCC', 'TREA', 'TRESPASS', 'VANDALISM', 'VEHICLE THEFT', 'WARRANTS', 'WEAPON LAWS']


In [36]:
print(PREDICTIONS_FILENAME)

predictions_20181108-175457.csv


In [37]:
df_test[submission_cols].to_csv(PREDICTIONS_FILENAME, index = False)